# Hyperparameter Tuning on Gradient Boosted Tree Model

Read the following documentation:
- https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html#for-better-accuracy 
- https://lightgbm.readthedocs.io/en/latest/Parameters.html


Then, run the code below

In [ ]:
import pandas as pd
import numpy as np

import lightgbm as lgbm
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report, recall_score

### Data Preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/dataset/cleaned_data.csv')
df['Attrition'] = [0 if x == 'No' else 1 for x in df['Attrition'].values]

categorical_variable = []

for x in df.columns:
    if df[x].dtype == 'O':
        categorical_variable.append(x)
        
df[categorical_variable] = df[categorical_variable].astype('category')

independent_var = df.drop('Attrition', axis = 'columns').columns.tolist()
target_var = ['Attrition']

X = df[independent_var]
y = df['Attrition']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

#!pip install imblearn

from imblearn.over_sampling import RandomOverSampler
X_train_sampled, y_train_sampled = RandomOverSampler(random_state = 42).fit_resample(X_train, y_train)

### **RandomizedSearch**

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

lgb_model = LGBMClassifier()
parameters = {
    'num_leaves':[10,20,40,60,80,100,120,150,170,200],
    'max_depth':[3,4,5,6,7,8,9,10,11,12],
    'min_data_in_leaf':[1,2,3,5,10,15,20,25,30],
    'boosting':['gbdt', 'dart']
}

lgb_search = RandomizedSearchCV(estimator = lgb_model, param_distributions = parameters, cv = 5, n_jobs = -1,
                              scoring = 'recall', n_iter = 100)

lgb_search.fit(X_train_sampled, y_train_sampled)
lgb_search.best_params_

{'num_leaves': 150, 'min_data_in_leaf': 3, 'max_depth': 12, 'boosting': 'dart'}

In [ ]:
search_model = LGBMClassifier(num_leaves= 80, min_data_in_leaf = 1, max_depth= 11, boosting= 'gbdt')

In [ ]:
search_model.fit(X_train_sampled, y_train_sampled)

LGBMClassifier(boosting='gbdt', max_depth=11, min_data_in_leaf=1, num_leaves=80)

In [ ]:
search_pred = search_model.predict(X_test)
search_recall = recall_score(y_test, search_pred)
print(search_recall)

0.8470588235294118


### **Default model**

In [ ]:
default_model = LGBMClassifier()
default_model.fit(X_train_sampled, y_train_sampled)

LGBMClassifier()

In [ ]:
def_pred = default_model.predict(X_test)
def_recall = recall_score(y_test, def_pred)
print(def_recall)

0.8588235294117647


### **1. Bayesian Optimization**

In [ ]:
pip install bayesian-optimization

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from bayes_opt import BayesianOptimization

#Prepare train set and test set
lgbmdata = lgbm.Dataset(data=X_train_sampled, label=y_train_sampled)

def cvlgbm(num_leaves, max_depth, min_data_in_leaf,
         learning_rate, bagging_freq):
  clf = LGBMClassifier(objective='binary',
                       num_leaves=round(num_leaves),
                       max_depth=round(max_depth),
                       min_data_in_leaf=round(min_data_in_leaf),
                       learning_rate=learning_rate,
                       bagging_freq = round(bagging_freq),
                       metric='recall',
                       eval_metric='recall',
                       verbose=-1)
  scores = cross_val_score(clf, X_train_sampled, y_train_sampled, cv=5, scoring='recall')
  return np.mean(scores)

In [ ]:
optimizer = BayesianOptimization(cvlgbm,
                                 {'num_leaves':(5,50),
                                  'max_depth':(-1,10),
                                  'min_data_in_leaf':(10,50),
                                  'learning_rate':(0.01,0.5),
                                  'bagging_freq':(0,10)},
                                 random_state=100,
                                 verbose=2)

In [ ]:
optimizer.maximize(init_points=2, n_iter=5)

|   iter    |  target   | baggin... | learni... | max_depth | min_da... | num_le... |
-------------------------------------------------------------------------------------
| 1         | 0.9048    | 5.434     | 0.1464    | 3.67      | 43.79     | 5.212     |
| 2         | 1.0       | 1.216     | 0.3387    | 8.084     | 15.47     | 30.88     |
| 3         | 0.9995    | 0.6936    | 0.2388    | 7.821     | 13.97     | 30.35     |
| 4         | 0.9995    | 10.0      | 0.5       | -1.0      | 10.0      | 50.0      |
| 5         | 0.9098    | 0.0       | 0.01      | 10.0      | 50.0      | 50.0      |
| 6         | 0.9482    | 0.0       | 0.01      | 10.0      | 10.0      | 50.0      |
| 7         | 0.9995    | 10.0      | 0.5       | -1.0      | 10.0      | 31.84     |


In [ ]:
best_params = optimizer.max['params']
best_params['min_data_in_leaf'] = int(np.floor(best_params['min_data_in_leaf']))
best_params['bagging_freq'] = int(np.floor(best_params['bagging_freq']))
best_params['num_leaves'] = int(np.floor(best_params['num_leaves']))
best_params['max_depth'] = int(np.floor(best_params['max_depth']))
best_params

{'bagging_freq': 1,
 'learning_rate': 0.33866705151612153,
 'max_depth': 8,
 'min_data_in_leaf': 15,
 'num_leaves': 30}

In [ ]:
tuned_clf = LGBMClassifier(**best_params)

In [ ]:
tuned_clf.fit(X_train_sampled, y_train_sampled)
pred = tuned_clf.predict(X_test)

In [ ]:
recall = recall_score(y_test, pred)
print(recall)

0.8764705882352941


### **2. Hyperopt**

In [ ]:
from hyperopt import tpe, hp
from hyperopt.fmin import fmin

In [ ]:
space = {'num_leaves':hp.quniform('num_leaves',5,50,1),
        'max_depth':hp.quniform('max_depth',-1,10,1),
        'min_data_in_leaf':hp.quniform('min_data_in_leaf',10,50,1),
        'learning_rate':hp.uniform('learning_rate',0.01,0.5),
        'bagging_freq':hp.quniform('bagging_freq',0,10,1)}

def recall(target, pred):
  return -recall_score(target, pred)

def cvlgbm(params):
  clf = LGBMClassifier(objective='binary',
                       num_leaves=int(params['num_leaves']),
                       max_depth=int(params['max_depth']),
                       min_data_in_leaf=int(params['min_data_in_leaf']),
                       learning_rate=params['learning_rate'],
                       bagging_freq = int(params['bagging_freq']),
                       metric='recall',
                       eval_metric='recall',
                       verbose=-1, random_state=100)
  scores = cross_val_score(clf, X_train_sampled, y_train_sampled, cv=5, scoring='recall', error_score='raise')
  return -np.mean(scores)

best = fmin(fn=cvlgbm, space=space, algo=tpe.suggest, max_evals=20)

 90%|█████████ | 18/20 [00:23<00:01,  1.04it/s, best loss: -0.9995012468827931]

In [ ]:
print(best)

{'bagging_freq': 8.0, 'learning_rate': 0.16825692602252965, 'max_depth': 10.0, 'min_data_in_leaf': 23.0, 'num_leaves': 43.0}


In [ ]:
hp_clf = LGBMClassifier(bagging_freq=8, learning_rate=0.16825692602252965, max_depth=10, min_data_in_leaf=23, num_leaves=43)

hp_clf.fit(X_train_sampled, y_train_sampled)
pred = hp_clf.predict(X_test)

recall = recall_score(y_test, pred)
print(recall)

0.8647058823529412


### **3. Optuna**

In [ ]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 9.6 MB/s eta 0:00:00


In [ ]:
import optuna

In [ ]:
def objective(trial):
  params = {'objective':'binary',
            'boosting':'gbdt',
            'metric':'recall',
            'random_state':100,
            'verbosity':-1,
            'num_leaves':trial.suggest_int('num_leaves',5,50),
            'max_depth':trial.suggest_int('max_depth',-1,10),
            'min_data_in_leaf':trial.suggest_int('min_data_in_leaf',10,50),
            'learning_rate':trial.suggest_float('learning_rate',0.01,0.5),
            'bagging_freq':trial.suggest_int('bagging_freq',0,10)}

  lgb = LGBMClassifier(**params)
  lgb.fit(X_train_sampled, y_train_sampled)
  pred = lgb.predict(X_test)
  recall = recall_score(y_test, pred)
  return recall

study_tuner = optuna.create_study(direction='maximize')

study_tuner.optimize(objective, n_trials=50)

[I 2023-03-07 14:02:54,750] A new study created in memory with name: no-name-8215f339-693c-400b-b2b2-0e90a8765471
[I 2023-03-07 14:02:55,173] Trial 0 finished with value: 0.8411764705882353 and parameters: {'num_leaves': 10, 'max_depth': 6, 'min_data_in_leaf': 14, 'learning_rate': 0.44354235099780437, 'bagging_freq': 6}. Best is trial 0 with value: 0.8411764705882353.
[I 2023-03-07 14:02:55,531] Trial 1 finished with value: 0.6647058823529411 and parameters: {'num_leaves': 10, 'max_depth': 4, 'min_data_in_leaf': 14, 'learning_rate': 0.020754165694198523, 'bagging_freq': 9}. Best is trial 0 with value: 0.8411764705882353.
[I 2023-03-07 14:02:55,670] Trial 2 finished with value: 0.8058823529411765 and parameters: {'num_leaves': 9, 'max_depth': 4, 'min_data_in_leaf': 15, 'learning_rate': 0.3946133016745912, 'bagging_freq': 8}. Best is trial 0 with value: 0.8411764705882353.
[I 2023-03-07 14:02:55,836] Trial 3 finished with value: 0.7705882352941177 and parameters: {'num_leaves': 10, 'max_

In [ ]:
best = study_tuner.best_params
print(best)

{'num_leaves': 31, 'max_depth': -1, 'min_data_in_leaf': 26, 'learning_rate': 0.4922187372632303, 'bagging_freq': 1}


In [ ]:
op_clf = LGBMClassifier(**best)

op_clf.fit(X_train_sampled, y_train_sampled)
pred = op_clf.predict(X_test)

recall = recall_score(y_test, pred)
print(recall)

0.8823529411764706


### **4. Ray Tune**

In [ ]:
pip install ray[tune]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from ray import air, tune
from ray.air import session
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.lightgbm import TuneReportCheckpointCallback

In [ ]:
def train_attrition(config):
  X_train, X_val, y_train, y_val = train_test_split(
    X_train_sampled, y_train_sampled, test_size=0.3, random_state=42)
  train_set = lgbm.Dataset(X_train, label=y_train)
  test_set = lgbm.Dataset(X_val, label=y_val)
  gbm = lgbm.train(config, train_set,
                   valid_sets=[test_set],
                   valid_names=['eval'],
                   verbose_eval=False,
                   callbacks=[TuneReportCheckpointCallback(
                       {'binary_error':'eval-binary_error',
                        'binary_logloss':'eval-binary_logloss'})]
                   ),
  pred = gbm.predict(test_set)
  pred_labels = np.rint(pred)
  session.report({'recall':recall_score(y_test, pred_labels),
                  'done':True})
  
config = {'objective':'binary',
            'boosting':'gbdt',
            'metric':['binary_error','binary_logloss'],
            'random_state':100,
            'verbosity':-1,
            'num_leaves':tune.randint(5,50),
            'max_depth':tune.randint(-1,10),
            'min_data_in_leaf':tune.randint(10,50),
            'learning_rate':tune.loguniform(0.01,0.5),
            'bagging_freq':tune.randint(0,10)}

tuner = tune.Tuner(train_attrition,
                   tune_config=tune.TuneConfig(
                       metric='binary_error',
                       mode='min',
                       scheduler=ASHAScheduler(),
                       num_samples=10),
                   param_space=config
                   )

In [ ]:
results = tuner.fit()

2023-03-08 10:22:10,123	INFO worker.py:1553 -- Started a local Ray instance.


(train_attrition pid=31270) /usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
(train_attrition pid=31270)   warnings.warn('categorical_feature in param dict is overridden.')


Trial name,binary_error,binary_logloss,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_attrition_18094_00000,0.2,0.49091,2023-03-08_10-22-40,True,,82abb0aa3b3e4cd595a571f6b7e0fff3,256b117648ba,100,172.28.0.12,31270,14.1078,0.00548291,14.1078,1678270960,0,,100,18094_00000,0.00805187
train_attrition_18094_00001,0.039834,0.205112,2023-03-08_10-22-40,True,,f26159d23cd14fafb2f07da86d2ec749,256b117648ba,100,172.28.0.12,31340,1.90472,0.033309,1.90472,1678270960,0,,100,18094_00001,0.00463963
train_attrition_18094_00002,0.221577,0.634087,2023-03-08_10-22-40,True,,82abb0aa3b3e4cd595a571f6b7e0fff3,256b117648ba,1,172.28.0.12,31270,0.0577185,0.0577185,0.0577185,1678270960,0,,1,18094_00002,0.00805187
train_attrition_18094_00003,0.208299,0.557248,2023-03-08_10-22-40,True,,82abb0aa3b3e4cd595a571f6b7e0fff3,256b117648ba,1,172.28.0.12,31270,0.0784388,0.0784388,0.0784388,1678270960,0,,1,18094_00003,0.00805187
train_attrition_18094_00004,0.212448,0.664281,2023-03-08_10-22-40,True,,82abb0aa3b3e4cd595a571f6b7e0fff3,256b117648ba,1,172.28.0.12,31270,0.0672958,0.0672958,0.0672958,1678270960,0,,1,18094_00004,0.00805187
train_attrition_18094_00005,0.304564,0.686552,2023-03-08_10-22-40,True,,82abb0aa3b3e4cd595a571f6b7e0fff3,256b117648ba,1,172.28.0.12,31270,0.0710759,0.0710759,0.0710759,1678270960,0,,1,18094_00005,0.00805187
train_attrition_18094_00006,0.217427,0.550696,2023-03-08_10-22-40,True,,82abb0aa3b3e4cd595a571f6b7e0fff3,256b117648ba,1,172.28.0.12,31270,0.048347,0.048347,0.048347,1678270960,0,,1,18094_00006,0.00805187
train_attrition_18094_00007,0.231535,0.682256,2023-03-08_10-22-40,True,,82abb0aa3b3e4cd595a571f6b7e0fff3,256b117648ba,1,172.28.0.12,31270,0.0638478,0.0638478,0.0638478,1678270960,0,,1,18094_00007,0.00805187
train_attrition_18094_00008,0.304564,0.687283,2023-03-08_10-22-40,True,,82abb0aa3b3e4cd595a571f6b7e0fff3,256b117648ba,1,172.28.0.12,31270,0.0581408,0.0581408,0.0581408,1678270960,0,,1,18094_00008,0.00805187
train_attrition_18094_00009,0.512033,0.690695,2023-03-08_10-22-40,True,,f26159d23cd14fafb2f07da86d2ec749,256b117648ba,1,172.28.0.12,31340,0.0667381,0.0667381,0.0667381,1678270960,0,,1,18094_00009,0.00463963


(train_attrition pid=31340) /usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
(train_attrition pid=31340)   warnings.warn('categorical_feature in param dict is overridden.')
2023-03-08 10:22:41,088	INFO tune.py:798 -- Total run time: 27.30 seconds (22.29 seconds for the tuning loop).


In [ ]:
best = results.get_best_result().config
print(best)

{'objective': 'binary', 'boosting': 'gbdt', 'metric': ['binary_error', 'binary_logloss'], 'random_state': 100, 'verbosity': -1, 'num_leaves': 41, 'max_depth': 0, 'min_data_in_leaf': 20, 'learning_rate': 0.03023228780231226, 'bagging_freq': 8}


In [ ]:
rt_clf = LGBMClassifier(bagging_freq=8, learning_rate=0.03023228780231226, max_depth=0, min_data_in_leaf=20, num_leaves=41)

rt_clf.fit(X_train_sampled, y_train_sampled)
pred = rt_clf.predict(X_test)

recall = recall_score(y_test, pred)
print(recall)

0.8176470588235294


### Answer the Questions Below

#### 1. Which model is better? The tuned model, or the default model? Explain why. 

The default model outperforms the tuned model using  RandomizedSearchCV. The tuned model's recall score may be lower because the parameter ranges are too narrow or do not include the default value, a value less than the default value, or a value greater than the default value.

#### 2. Coba cari tahu di internet, apakah ada cara-cara tertentu untuk melakukan Hyperparameter Tuning secara automated sehingga lebih baik daripada cara kita melakukan Randomized Search CV ini? Jika ya, sebutkan cara-cara melakukan Hyperparameter Tuning dengan cara yang lebih baik daripadda Randomized Search CV ini. 

Bayesian Optimization (https://github.com/fmfn/BayesianOptimization), Hyperopt (https://github.com/hyperopt/hyperopt), and Optuna (https://optuna.org/) gave better result than Randomized Search CV.

#### 3. Jika model underfit mapun overfit. Apakah Hyperparameter Tuning adalah solusi utama untuk menyelesaikan persoalan tersebut?

Hyperparameter tuning is one method for dealing with overfitting and underfitting in order to reduce or increase the model's complexity. Other methods for dealing with overfitting and underfitting include adding features and increasing training duration.